In [ ]:
!pip install openai
!pip install sentence_transformers


import pandas as pd
import os

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## Create dataframes and add context embeddings

In [ ]:
data_path="/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_context_emotion_prop2.csv"

test_df = pd.read_csv(data_path)

In [ ]:
PATH="/content/drive/MyDrive/semester-project-propaganda_umberto/code/data"

train_dir = PATH + "/train/"

train_df = pd.DataFrame(columns=["id", "text"])

val_dir = PATH + "/dev/"
val_df = pd.DataFrame(columns=["id", "text"])

test_dir = PATH + "/test/"
test_df = pd.DataFrame(columns=["id", "text"])


# Reading of the .txt files that contains only the id and the text.
for filename in os.listdir(train_dir):
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(train_dir + filename, "r") as f:
            content = f.read()
        train_df = train_df.append({"id": id, "text": content}, ignore_index=True)


# Reading of the .txt files that contains only the id and the text.
for filename in os.listdir(val_dir):
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(val_dir + filename, "r") as f:
            content = f.read()
        val_df = val_df.append({"id": id, "text": content}, ignore_index=True)

# Reading of the .txt files that contains only the id and the text.
for filename in os.listdir(test_dir):
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(test_dir + filename, "r") as f:
            content = f.read()
        test_df = test_df.append({"id": id, "text": content}, ignore_index=True)

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch
from sentence_transformers import SentenceTransformer

# Load the Sentence-BERT model
# all-mpnet-base-v2 -> sbert
model = SentenceTransformer('all-mpnet-base-v2')

# Define a function to get the sentence encoding using Sentence-BERT
def get_sentence_encoding(sentence):
    # Encode the sentence
    sentence_encoding = model.encode([sentence])

    return sentence_encoding

In [ ]:
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import warnings
warnings.filterwarnings("ignore")


def split_into_sentences2(text):
  fixed_text = text.replace('”', '"').replace('?"', '"?').replace('!"', '"!').replace('."', '".').replace('.’"', '’".').replace('.’', '’.')
  sent_text = nltk.sent_tokenize(fixed_text) # this gives us a list of sentences
  return sent_text  

import re
import pandas as pd
import os

train_df = pd.DataFrame(columns=['sentence_id', 'doc_id', 'start', 'end', 'text', 'label'])

sent_n = 0
for i, filename in enumerate(os.listdir(train_dir)):
    print(f"{i}/{len(os.listdir(train_dir))}")
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(train_dir + filename, "r") as f:
            content = f.read()
            sentences = split_into_sentences2(content)
            sent_n = sent_n + len(sentences)
            
            output = []
            start = 1
            lines = content.split("\n")
            title = lines[0]
            # print(content)
            # print(f"Title: {title}")
            title_embedding =  get_sentence_encoding(title)

            for i, sentence in enumerate(sentences):
                sentence_before = ""
                sentence_after = ""
                sentence_before_encoding = None
                sentence_after_encoding = None
                if i > 0:
                  sentence_before = sentences[i-1]
                  sentence_before_encoding = get_sentence_encoding(sentence_before)
                if i < (len(sentences)-1):
                  sentence_after = sentences[i+1]
                  sentence_after_encoding = get_sentence_encoding(sentence_after)
                sentence_encoding = get_sentence_encoding(sentence)
                # iterate over each sentence and calculate the starting and ending positions
                start = content.find(sentence, start)
                end = start + len(sentence) - 1
                #df["id"] = df.index + 1
                train_df = train_df.append({'doc_id' : id, 'start' : int(start), 'end' : int(end), 'text' : sentence.strip(), 'text_before': sentence_before.strip(), 'text_after': sentence_after.strip(), 'label' : 'None', 'sentence_encoding': sentence_encoding, 'sentence_before_encoding': sentence_before_encoding, 'sentence_after_encoding': sentence_after_encoding, "title": title, "title_embedding":title_embedding}, ignore_index = True)
                start = end + 1

train_df = train_df.astype({'start':int, 'end':int})


In [ ]:

# PATH="/content/drive/MyDrive/semester-project-propaganda_umberto/code/data"
train_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/train_df_context.csv', index=False)


In [ ]:
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import warnings
warnings.filterwarnings("ignore")


def split_into_sentences2(text):
  fixed_text = text.replace('”', '"').replace('?"', '"?').replace('!"', '"!').replace('."', '".').replace('.’"', '’".').replace('.’', '’.')
  sent_text = nltk.sent_tokenize(fixed_text) # this gives us a list of sentences
  return sent_text  

import re
import pandas as pd
import os

val_df = pd.DataFrame(columns=['sentence_id', 'doc_id', 'start', 'end', 'text', 'label'])

sent_n = 0
for i, filename in enumerate(os.listdir(val_dir)):
    print(f"{i}/{len(os.listdir(val_dir))}")
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(val_dir + filename, "r") as f:
            content = f.read()
            sentences = split_into_sentences2(content)
            sent_n = sent_n + len(sentences)
            
            output = []
            start = 1
            lines = content.split("\n")
            title = lines[0]
            # print(content)
            # print(f"Title: {title}")
            title_embedding =  get_sentence_encoding(title)

            for i, sentence in enumerate(sentences):
                sentence_before = ""
                sentence_after = ""
                sentence_before_encoding = None
                sentence_after_encoding = None
                if i > 0:
                  sentence_before = sentences[i-1]
                  sentence_before_encoding = get_sentence_encoding(sentence_before)
                if i < (len(sentences)-1):
                  sentence_after = sentences[i+1]
                  sentence_after_encoding = get_sentence_encoding(sentence_after)
                sentence_encoding = get_sentence_encoding(sentence)
                # iterate over each sentence and calculate the starting and ending positions
                start = content.find(sentence, start)
                end = start + len(sentence) - 1
                #df["id"] = df.index + 1
                val_df = val_df.append({'doc_id' : id, 'start' : int(start), 'end' : int(end), 'text' : sentence.strip(), 'text_before': sentence_before.strip(), 'text_after': sentence_after.strip(), 'label' : 'None', 'sentence_encoding': sentence_encoding, 'sentence_before_encoding': sentence_before_encoding, 'sentence_after_encoding': sentence_after_encoding, "title": title, "title_embedding":title_embedding}, ignore_index = True)
                start = end + 1

val_df = val_df.astype({'start':int, 'end':int})


In [ ]:
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import warnings
warnings.filterwarnings("ignore")


def split_into_sentences2(text):
  fixed_text = text.replace('”', '"').replace('?"', '"?').replace('!"', '"!').replace('."', '".').replace('.’"', '’".').replace('.’', '’.')
  sent_text = nltk.sent_tokenize(fixed_text) # this gives us a list of sentences
  return sent_text  

import re
import pandas as pd
import os

test_df = pd.DataFrame(columns=['sentence_id', 'doc_id', 'start', 'end', 'text', 'label'])

sent_n = 0
for i, filename in enumerate(os.listdir(test_dir)):
    print(f"{i}/{len(os.listdir(test_dir))}")
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(test_dir + filename, "r") as f:
            content = f.read()
            sentences = split_into_sentences2(content)
            sent_n = sent_n + len(sentences)
            
            output = []
            start = 1
            lines = content.split("\n")
            title = lines[0]
            # print(content)
            # print(f"Title: {title}")
            title_embedding =  get_sentence_encoding(title)

            for i, sentence in enumerate(sentences):
                sentence_before = ""
                sentence_after = ""
                sentence_before_encoding = None
                sentence_after_encoding = None
                if i > 0:
                  sentence_before = sentences[i-1]
                  sentence_before_encoding = get_sentence_encoding(sentence_before)
                if i < (len(sentences)-1):
                  sentence_after = sentences[i+1]
                  sentence_after_encoding = get_sentence_encoding(sentence_after)
                sentence_encoding = get_sentence_encoding(sentence)
                # iterate over each sentence and calculate the starting and ending positions
                start = content.find(sentence, start)
                end = start + len(sentence) - 1
                #df["id"] = df.index + 1
                test_df = test_df.append({'doc_id' : id, 'start' : int(start), 'end' : int(end), 'text' : sentence.strip(), 'text_before': sentence_before.strip(), 'text_after': sentence_after.strip(), 'label' : 'None', 'sentence_encoding': sentence_encoding, 'sentence_before_encoding': sentence_before_encoding, 'sentence_after_encoding': sentence_after_encoding, "title": title, "title_embedding":title_embedding}, ignore_index = True)
                start = end + 1

test_df = test_df.astype({'start':int, 'end':int})
test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_context.csv', index=False)


In [ ]:
import re
import pandas as pd
import os

val_df = pd.DataFrame(columns=['sentence_id', 'doc_id', 'start', 'end', 'text', 'label'])

sent_n = 0
for filename in os.listdir(val_dir):
    
    if filename.endswith(".txt"):
        id = filename.split(".")[0][7:]
        with open(val_dir + filename, "r") as f:
            content = f.read()
            sentences = split_into_sentences2(content)
            sent_n = sent_n + len(sentences)
            
            output = []
            start = 1

            for sentence in sentences:
                # iterate over each sentence and calculate the starting and ending positions
                start = content.find(sentence, start)
                end = start + len(sentence) - 1
                #df["id"] = df.index + 1
                val_df = val_df.append({'doc_id' : id, 'start' : int(start), 'end' : int(end), 'text' : sentence.strip(), 'label' : 'None'}, ignore_index = True)
                start = end + 1

val_df = val_df.astype({'start':int, 'end':int})


In [ ]:
label_dict = {
    'None' : 0,
    'Whataboutism' : 1,
    'Causal_Oversimplification' : 2,
    'Exaggeration,Minimisation' : 3, 
    'Doubt' : 4, 
    'Loaded_Language' : 5,
    'Name_Calling,Labeling' : 6, 
    'Flag-Waving' : 7, 
    'Reductio_ad_hitlerum' : 8,
    'Slogans' : 9, 
    'Appeal_to_fear-prejudice' : 10, 
    'Repetition' : 11,
    'Thought-terminating_Cliches' : 12, 
    'Black-and-White_Fallacy' : 13,
    'Bandwagon' : 14, 
    'Appeal_to_Authority' : 15, 
    'Red_Herring' : 16,
    'Obfuscation,Intentional_Vagueness,Confusion' : 17, 
    'Straw_Men' : 18
}

id_dict = {
    0:'None',
    1:'Whataboutism',
    2:'Causal_Oversimplification',
    3:'Exaggeration,Minimisation', 
    4:'Doubt', 
    5:'Loaded_Language',
    6:'Name_Calling,Labeling', 
    7:'Flag-Waving', 
    8:'Reductio_ad_hitlerum',
    9:'Slogans', 
    10:'Appeal_to_fear-prejudice', 
    11:'Repetition',
    12:'Thought-terminating_Cliches', 
    13:'Black-and-White_Fallacy',
    14:'Bandwagon', 
    15:'Appeal_to_Authority', 
    16:'Red_Herring',
    17:'Obfuscation,Intentional_Vagueness,Confusion', 
    18:'Straw_Men'
}



def label_name_to_id(text):
  labels = ""
  first = True
  for key in label_dict:
    if key in text:
      if not first:
        labels += ","
        print(key)
      labels += str(label_dict[key])
      first = False
  return labels

print(label_name_to_id("None"))


def id_to_label(text):
  labels = ""
  first = True
  arr = text.split(",")
  for key in arr:
    if not first:
      labels += ","
      # print(key)
    labels += id_dict[int(key)]
    first = False
  return labels

id_to_label("1,2,3")


In [ ]:
import numpy as np
import random

train_df_copy = train_df.copy()
val_df_copy = val_df.copy()


technique_sizes = [[] for i in range (len(id_dict))]


for filename in os.listdir(train_dir):
    if filename.endswith(".tsv"):
        tmp_df = pd.read_csv(train_dir + filename, sep='\t', names = ['id', 'type', 'start', 'end'])
        id = filename.split(".")[0][7:]
        
        #tmp_df = tmp_df.astype({'start' : int, 'end' : int})

        for _, row in tmp_df.iterrows():
          tech_id = label_dict[row["type"]]
          technique_sizes[tech_id].append(int(row['end']) - int(row['start']))
          v = train_df_copy.loc[(train_df_copy['doc_id'] == id) & 
                           ((train_df_copy['start'] <= row['start']) & #p starts after beginning of the sentence and before the ending
                           (train_df_copy['end'] >= row['start']) |
                           (train_df_copy['start'] <= row['end']) & #p starts after beginning of the sentence and before the ending
                           (train_df_copy['end'] >= row['end']))] 

          

          if len(train_df_copy.loc[v.index, 'label'].values) == 0:
            continue
          current_label = train_df_copy.loc[v.index, 'label'].values[0]
          if current_label == "None":
            train_df_copy.loc[v.index, 'label'] = str(label_dict[row["type"]])
          else:
            train_df_copy.loc[v.index, 'label'] = str(current_label) + "," + str(label_dict[row["type"]])
        
import numpy as np
import random

technique_sizes_val = [[] for i in range (len(id_dict))]


for filename in os.listdir(val_dir):
    if filename.endswith(".tsv"):
        tmp_df = pd.read_csv(val_dir + filename, sep='\t', names = ['id', 'type', 'start', 'end'])
        id = filename.split(".")[0][7:]
        
        #tmp_df = tmp_df.astype({'start' : int, 'end' : int})

        for _, row in tmp_df.iterrows():
          tech_id = label_dict[row["type"]]
          technique_sizes_val[tech_id].append(int(row['end']) - int(row['start']))
          v = val_df_copy.loc[(val_df_copy['doc_id'] == id) & 
                           ((val_df_copy['start'] <= row['start']) & #p starts after beginning of the sentence and before the ending
                           (val_df_copy['end'] >= row['start']) |
                           (val_df_copy['start'] <= row['end']) & #p starts after beginning of the sentence and before the ending
                           (val_df_copy['end'] >= row['end']))] 

          

          if len(val_df_copy.loc[v.index, 'label'].values) == 0:
            continue
          current_label = val_df_copy.loc[v.index, 'label'].values[0]
          if current_label == "None":
            val_df_copy.loc[v.index, 'label'] = str(label_dict[row["type"]])
          else:
            val_df_copy.loc[v.index, 'label'] = str(current_label) + "," + str(label_dict[row["type"]])
  


In [ ]:
technique_sizes_test = [[] for i in range (len(id_dict))]

for filename in os.listdir(test_dir):
    if filename.endswith(".tsv"):
        tmp_df = pd.read_csv(test_dir + filename, sep='\t', names = ['id', 'type', 'start', 'end'])
        id = filename.split(".")[0][7:]
        
        #tmp_df = tmp_df.astype({'start' : int, 'end' : int})

        for _, row in tmp_df.iterrows():
          tech_id = label_dict[row["type"]]
          technique_sizes_test[tech_id].append(int(row['end']) - int(row['start']))
          v = test_df.loc[(test_df['doc_id'] == id) & 
                           ((test_df['start'] <= row['start']) & #p starts after beginning of the sentence and before the ending
                           (test_df['end'] >= row['start']) |
                           (test_df['start'] <= row['end']) & #p starts after beginning of the sentence and before the ending
                           (test_df['end'] >= row['end']))] 

          

          if len(test_df.loc[v.index, 'label'].values) == 0:
            continue
          current_label = test_df.loc[v.index, 'label'].values[0]
          if current_label == "None":
            test_df.loc[v.index, 'label'] = str(label_dict[row["type"]])
          else:
            test_df.loc[v.index, 'label'] = str(current_label) + "," + str(label_dict[row["type"]])
   

## Add emotion features

In [ ]:

!pip install openai
!pip install sentence_transformers


import pandas as pd
import os

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import pandas as pd
data_path="/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/train_df_context.csv"

train_df_copy = pd.read_csv(data_path)

train_df_copy

In [ ]:
emotions = [
    "Fear",
    "Anger",
    "Anxiety",
    "Suspicion",
    "Hatred",
    "Paranoia",
    "Resentment",
    "Hostility",
    "Disgust",
    "Contempt",
    "Bitterness",
    "Cynicism",
    "Apathy",
    "Despair",
    "Pessimism",
    "Resignation",
    "Sorrow",
    "Regret",
    "Guilt",
    "Shame",
    "Embarrassment",
    "Humiliation",
    "Insecurity",
    "Vulnerability",
    "Intimidation",
    "Worry",
    "Panic",
    "Confusion",
    "Distress",
    "Amusement",
    "Joy",
    "Delight",
    "Excitement",
    "Satisfaction",
    "Pride",
    "Compassion",
    "Empathy",
    "Trust",
    "Love",
    "Affection",
    "Admiration",
    "Respect",
    "Hope",
    "Optimism",
    "Enthusiasm",
    "Inspiration",
    "Motivation",
    "Fascination",
    "Surprise",
    "Excitement",
    "Regard",
    "Belongingness",
    "Patriotism",
    "Pride",
    "Unity",
    "Authority",
    "Prejudice",
    "Doubt",
    "Nationalism"
]


In [ ]:
import openai

# Set up your OpenAI API credentials
openai.api_key = 'api'

def get_vector(tokens, probs):
  # arr = np.zeros(shape=(len(emotions),))
  arr = [0 for i in range(len(emotions))]
  for i, token in enumerate(tokens):
    token = token.strip()
    prob = probs[i].strip()
    try:
      ind = emotions.index(token)
      arr[ind] = prob
    except Exception as e:
    # Handling the exception
      print("An exception occurred:", e)
  return arr


# Define a function to perform multi-class classification
def detect_emotions(text, emotions):
    # Prompt for the classification task
    # prompt = "This is an emotion detection model specifically trained to analyze text for propaganda-related content. It is designed for multi-class text classification. Please provide some text for emotion detection:\n\nText: " + text + "\n\nOptions: " + ', '.join(emotions) + "\n\nClasses:"

        # Define the system message
    system_msg = 'You are a helpful assistant who performs emotion detection specifically trained to analyze text for propaganda-related content. You are designed for multi-class text classification. For each given sentence return detected emotions from the possible options and also return the probability of each emotion in form of a python array. '

    # Define the user message
    user_msg = "Detect emotions in next sentence: \"In the course of their conversations, Patel discussed his desire to see a holy war between Muslims and non-Muslims.\""+ "\nOptions: " + ', '.join(emotions)

    assistant_msg = "Anger, Hostility, Paranoia, Worry\nProbabilities: [0.54, 0.34, 0.11, 0.05]"

    try:
    # Create a dataset using GPT
      response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                              messages=[
          {"role": "system", "content": system_msg},
          {"role": "user", "content": user_msg},
          {"role": "assistant", "content": assistant_msg},
          {"role": "user", "content": "Detect emotions in the next sentence: \n\""+text+"\"\nOptions: "+ ', '.join(emotions)}
      ])

      # Extract the generated classifications from the response
      # token_logprobs = response.choices[0].logprobs.token_logprobs
      tokens = response.choices[0].message.content.split("\n")[0].split(",")
      probs = response.choices[0].message.content.split("Probabilities:")[1].replace("[", "").replace("]", "").split(",")
      vec = get_vector(tokens, probs)
      return vec
    except Exception as e:
    # Handling the exception
      print("An exception occurred:", e)
      # print(response)
# Example usage
text_to_classify = "To no one’s surprise, Trump’s legal tormentor, Stormy Daniels attorney Michael Avenatti, whom Tucker Carlson has dubbed \“creepy porn lawyer,\" is representing Swetnick."

# class_options = ['Option 1', 'Option 2', 'Option 3']  # List of possible classes


# Perform multi-class classification
classified_labels = detect_emotions(text_to_classify, emotions)
print("Classified labels:", classified_labels)

In [ ]:
test_df = test_df.assign(emotions_vector=pd.Series(dtype=object))

In [ ]:
import math

for index, row in test_df.iterrows():
    print(f"{index}/{test_df.shape[0]}")
    if isinstance(row["emotions_vector"], float) and math.isnan(row["emotions_vector"]) or row["emotions_vector"]=="nan":
        try:
          if index % 1000 == 0:
            test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_ontext_emotion.csv', index=False)

          value = detect_emotions(row["text"], emotions)
          print(f"Vector is: {value}")
          test_df.at[index, 'emotions_vector'] =  value
        except Exception as e:
          # Handling the exception
          print("An exception occurred:", e)

test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_ontext_emotion.csv', index=False)


In [ ]:
train_df_context = train_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/train_df_context.csv', index=False)


In [ ]:
#add emotions

train_df_copy = train_df.copy()

for index, row in train_df.iterrows():
    train_df_copy.at[index, 'emotions'] = new_column_value
    train_df_copy.at[index, 'emotions_encoding'] = new_column_value

## Add propaganda techniques

In [ ]:
propaganda_techniques = ['Whataboutism',
    'Causal_Oversimplification',
    'Exaggeration_Minimisation', 
    'Doubt', 
    'Loaded_Language',
    'Name_Calling_Labeling', 
    'Flag-Waving', 
    'Reductio_ad_hitlerum',
    'Slogans', 
    'Appeal_to_fear-prejudice', 
    'Repetition',
    'Thought-terminating_Cliches', 
    'Black-and-White_Fallacy',
    'Bandwagon', 
    'Appeal_to_Authority', 
    'Red_Herring',
    'Obfuscation', 
    'Straw_Men'
]

In [ ]:
instructions = ['1. Whataboutism:  Discredit an opponent’s position by charging them with hypocrisy without directly disproving their argument (Richter, 2017). For example, mentioning an event that discredits the opponent: “What about . . . ?” (Richter, 2017). \n examples: \"President Trump —who himself avoided national military service in the 1960’s— keeps beating the war drums over North Korea.\" ; "Russia Today had a proclivity for whataboutism in its coverage of the 2015 Baltimore and Ferguson protests in the US, which revealed a consistent refrain: “the oppression of blacks in the US has become so unbearable that the eruption of violence was inevitable”, and that the US therefore lacks “the moral high ground to discuss human rights issues in countries like Russia and China.”"',
    '2. Causal_Oversimplification: Assuming a single cause or reason when there are multiple causes behind an issue. We include in the definition also scapegoating, i.e. the transfer of the blame to one person or group of people without investigating the complexities of an issue.\n example: \"If France had not have declared war on Germany then World War II would have never happened.\"',
    '3. Exaggeration_Minimisation: Either representing something in an excessive manner: making things larger, better, worse or making something seem less important or smaller than it actually is, e.g., saying that an insult was just a joke (Jowett and O’Donnell, 2012b, pag. 303).\n examples: \"Coronavirus ‘risk to the American people remains very low’, Trump said.\", “Democrats bolted as soon as Trump’s speech ended in an apparent effort to signal they can’t even stomach being in the same room as the president”; “I was not fighting with her; we were just playing.”', 
    '4. Doubt:  Questioning the credibility of someone or something.\n examples: \"Can the same be said for the Obama Administration?\"; A candidate says about his opponent: “Is he ready to be the Mayor?”', 
    '5. Loaded_Language: Using specific words and phrases with strong emotional implications (either positive or negative) to influence an audience (Weston, 2000, p. 6).\n Examples: \"Outrage as Donald Trump suggests injecting disinfectant to kill virus.\"; “[. . . ] a lone lawmaker’s childish shouting.”',
    '6. Name_Calling_Labeling: Labeling the object of the propaganda campaign as either something the target audience fears, hates, finds undesirable or loves, praises (Miller, 1939).\n Examples:  “Republican congressweasels”; “Bush the Leser.”;  \"WHO: Coronavirus emergency is ’Public Enemy Number 1’\"', 
    '7. Flag-Waving:  Playing on strong national feeling (or with respect to any group, e.g., race, gender, political preference) to justify or promote an action or idea (Hobbs and Mcgee, 2008).\n examples: \"Mueller attempts to stop the will of We the People!!! It’s time to jail Mueller.\"; “entering this war will make us have a better future in our country.”', 
    '8. Reductio_ad_hitlerum:  Persuading an audience to disapprove an action or idea by suggesting that it is popular with groups hated in contempt by the target audience. It can refer to any person or concept with a negative connotation (Teninbaum, 2009).\n examples: “Only one kind of person can think this way: a communist.”; \"“Vichy journalism,” a term which now fits so much of the mainstream media. It collaborates in the same way that the Vichy government in France collaborated with the Nazis.\"',
    '9. Slogans:  A brief and striking phrase that may include labeling and stereotyping. Slogans tend to act as emotional appeals (Dan, 2015).\n examples: \"“BUILD THE WALL!” Trump tweeted.\"; “Make America great again!”', 
    '10. Appeal_to_fear-prejudice: Seeking to build support for an idea by instilling anxiety and/or panic in the population towards an alternative, possibly based on preconceived judgments.\n examples: \"A dark, impenetrable and “irreversible” winter of persecution of the faithful by their own shepherds will fall.\"; “stop those refugees; they are terrorists.”', 
    '11. Repetition: Repeating the same message over and over again, so that the audience will eventually accept it (Torok, 2015; Miller, 1939). \n example: \"I still have a dream. It is a dream deeply rooted in the American dream. I have a dream that one day . . .\"',
    '12. Thought-terminating_Cliches:  Words or phrases that discourage critical thought and meaningful discussion on a topic. They are typically short, generic sentences that offer seemingly simple answers to complex questions or that distract attention away from other lines of thought (Hunter, 2015, p. 78).\n example: \"I do not really see any problems there. Marx is the President.\"; “it is what it is”; “you cannot judge it without experiencing it”; “it’s common sense”, “nothing is permanent except change”, “better late than never”; “mind your own business”; “nobody’s perfect”; “it doesn’t matter”; “you can’t change human nature.”', 
    '13. Black-and-White_Fallacy:  Presenting two alternative options as the only possibilities, when in fact more possibilities exist (Torok, 2015). Dictatorship is an extreme case: telling the audience exactly what actions to take, eliminating any other possible choice. \n examples: \" Francis said these words: “Everyone is guilty for the good he could have done and did not do . . . If we do not oppose evil, we tacitly feed it.”\"; “You must be a Republican or Democrat; you are not a Democrat. Therefore, you must be a Republican”; “There is no alternative to war.”',
    '14. Bandwagon:  Attempting to persuade the target audience to join in and take the course of action because “everyone else is taking the same action” (Hobbs and Mcgee, 2008).\n example: \"He tweeted, “EU no longer considers #Hamas a terrorist group. Time for US to do same.”\"; “Would you vote for Clinton as president? 57% say yes.”', 
    '15. Appeal_to_Authority:  Stating that a claim is true simply because a valid authority or expert on the issue supports it, without any other supporting evidence (Goodwin, 2011). We include in this technique the special case in which the reference is not an authority or an expert, although it is referred to as testimonial in the literature (Jowett and O’Donnell, 2012b, pag. 237).\n example: \"Monsignor Jean-Franois Lantheaume, who served as first Counsellor of the Nunciature in Washington, confirmed that “Vigan said the truth. That’s all.”\"', 
    '16. Red_Herring:Introducing irrelevant material to the issue being discussed, so that everyone’s attention is diverted away from the points made (Weston, 2018, p. 78). Those subjected to a red herring argument are led away from the issue that had been the focus of the discussion and urged to follow an observation or claim that may be associated with the original claim, but is not highly relevant to the issue in dispute (Teninbaum, 2009).\n examples: “You may claim that the death penalty is an ineffective deterrent against crime – but what about the victims of crime? How do you think surviving family members feel when they see the man who murdered their son kept in prison at their expense? Is it right that they should pay for their son’s murderer to be fed and housed?”; \"You may claim that the death penalty is an ineffective deterrent against crime – but what about the victims of crime? How do you think surviving family members feel when they see the man who murdered their son kept in prison at their expense? Is it right that they should pay for their son’s murderer to be fed and housed?”\"',
    '17. Obfuscation: Using deliberately unclear words, so that the audience may have its own interpretation (Suprabandari, 2007; Weston, 2018, p. 8). For instance, when an unclear phrase with multiple possible meanings is used within the argument, and, therefore, it does not really support the conclusion.\n examples: “It is a good idea to listen to victims of theft. Therefore, if the victims say to have the thief shot, then you should do it.”; \"The cardinal’s office maintains that rather than saying “yes,” there is a possibility of liturgical “blessing” of gay unions, he answered the question in a more subtle way without giving an explicit “yes.”\"', 
    '18. Straw_Men:  When an opponent’s proposition is substituted with a similar one which is then refuted in place of the original (Walton, 2013). Weston (2000, p. 78) specifies the characteristics of the substituted proposition: “caricaturing an opposing view so that it is easy to refute”.\n example: \"“Take it seriously, but with a large grain of salt.” Which is just Allen’s more nuanced way of saying: “Don’t believe it.”\"'
]

for i in instructions:
  print(i)
  print()



In [ ]:
import openai
import time

# Set up your OpenAI API credentials
openai.api_key = 'api'

def get_vector(tokens, probs):
  # arr = np.zeros(shape=(len(emotions),))
  arr = [0 for i in range(len(propaganda_techniques))]
  if len(probs)==1:
    print("just one")
    ind = propaganda_techniques.index(tokens[0])
    arr[ind] = probs[0]
    return arr

  for i, token in enumerate(tokens):
    token = token.strip()
    prob = probs[i].strip()
    try:
      ind = propaganda_techniques.index(token)
      arr[ind] = prob
    except Exception as e:
    # Handling the exception
      print("An exception occurred:", e)
  return arr


# Define a function to perform multi-class classification
def detect_techniques(text, propaganda_techniques):
    # Prompt for the classification task
    # prompt = "This is an emotion detection model specifically trained to analyze text for propaganda-related content. It is designed for multi-class text classification. Please provide some text for emotion detection:\n\nText: " + text + "\n\nOptions: " + ', '.join(emotions) + "\n\nClasses:"
    time.sleep(0.7)
        # Define the system message
    system_msg = 'You are a helpful assistant who performs multi label classification for propaganda techniques. For each given sentence return detected propaganda techniques from the possible options and also return the probability of each technique in a form of a python array. Possible propaganda techniques with explanations and examples: \n' + '\n '.join(instructions) + "\nPlease make sure you write the name of the classes exactly as in the list I provided."

    # Define the user message
    user_msg = "Detect propaganda techniques in following sentence: \" Instead, the tough-talking politician said they were regarded as “Muslim invaders\"."+ "\nOptions: " + ', '.join(propaganda_techniques) + " \nPlease make sure you write the name of the classes exactly as in the list I provided. If you don't detect any propaganda technique, please write exactly: \"No detected propaganda in this sentence.\""

    # assistant_msg = "Anger, Hostility, Paranoia, Worry\nProbabilities: [0.54, 0.34, 0.11, 0.05]"
    assistant_msg = "Name_Calling_Labeling, Appeal_to_Authority, Appeal_to_fear-prejudice, Loaded_Language\nProbabilities: [0.84, 0.09, 0.67, 0.87]"
    try:
    # Create a dataset using GPT
      response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                              messages=[
          {"role": "system", "content": system_msg},
          {"role": "user", "content": user_msg},
          {"role": "assistant", "content": assistant_msg},
          {"role": "user", "content": "Detect propaganda techniques in the next sentence: \n\""+text+"\"\nOptions: "+ ', '.join(propaganda_techniques)}
      ])

      print(response.choices[0].message.content)
      # print(type(response.choices[0].message.content))
      mess = response.choices[0].message.content
      if "No detected propaganda" in mess:
        return [0 for k in range(18)]

      # Extract the generated classifications from the response
      # token_logprobs = response.choices[0].logprobs.token_logprobs
      tokens = response.choices[0].message.content.split("\n")[0].split(",")
      try:        
        probs = response.choices[0].message.content.split("Probabilities:")[1].replace("[", "").replace("]", "").split(",")
      except:
        probs = response.choices[0].message.content.split("Probability:")[1].replace("[", "").replace("]", "").split(",")
      print(f"tokens: {tokens}")
      print(type(tokens))
      print(probs)
      vec = get_vector(tokens, probs)
      return vec
    except Exception as e:
    # Handling the exception
      print("An exception occurred:", e)
      # print(response)
# Example usage
text_to_classify = "To no one’s surprise, Trump’s legal tormentor, Stormy Daniels attorney Michael Avenatti, whom Tucker Carlson has dubbed \“creepy porn lawyer,\" is representing Swetnick."
# text_to_classify = "Hi, im Jana"
# class_options = ['Option 1', 'Option 2', 'Option 3']  # List of possible classes


# Perform multi-class classification
classified_labels = detect_techniques(text_to_classify, propaganda_techniques)
print("Classified labels:", classified_labels)

In [ ]:
test_df = test_df.assign(propaganda_vector=pd.Series(dtype=object))


In [ ]:
import math

for index, row in test_df.iterrows():
    print(f"{index}/{test_df.shape[0]}")
    if(isinstance(row["propaganda_vector"], float) and math.isnan(row["propaganda_vector"]) or row["propaganda_vector"]=="None" or row["propaganda_vector"] is None):
      try:
        if index % 1000 == 0 and index != 0:
          test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_context_emotion_prop.csv', index=False)

        value = detect_techniques(row["text"], propaganda_techniques)
        print(f"Vector is: {value}")
        test_df.at[index, 'propaganda_vector'] =  value
      except Exception as e:
        # Handling the exception
        print("An exception occurred:", e)

test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_context_emotion_prop.csv', index=False)


In [ ]:
test_df.to_csv('/content/drive/MyDrive/semester-project-propaganda_umberto/code/data/test_df_context_emotion_prop.csv', index=False)


In [ ]:
import openai

# Set up your OpenAI API credentials
openai.api_key = 'api'

# Define a function to perform multi-class classification
def detect_techniques(text, techniques):
    # Prompt for the classification task
    prompt = "This is model for recognizing propaganda techniques in a sentence. It is designed for multi-class text classification. Please provide some text for propaganda technique detection:\n\nText: " + text + "\n\nOptions: " + ', '.join(techniques) + "\n\nClasses:"

    # Generate a response using OpenAI GPT-3
    response = openai.Completion.create(
        engine='text-davinci-003',  # Specify the GPT-3 model to use
        prompt=prompt,
        temperature=0.5,  # Controls the randomness of the response
        max_tokens=len(techniques),  # Generate tokens for each class option
        # logprobs = 0.5,
        n=1  # Generate a single response
    )

    # Extract the generated classifications from the response
    print(response.choices)
    classifications = response.choices[0].text.strip().split("\n")

    return classifications

# Example usage
text_to_classify = "In the course of their conversations, Patel discussed his desire to see a holy war between Muslims and non-Muslims."
# class_options = ['Option 1', 'Option 2', 'Option 3']  # List of possible classes


# Perform multi-class classification
classified_labels = detect_techniques(text_to_classify, propaganda_techniques)
print("Classified labels:", classified_labels)

In [ ]:
#add techniques

train_df_copy = train_df.copy()

for index, row in train_df.iterrows():
    train_df_copy.at[index, 'emotions'] = new_column_value
    train_df_copy.at[index, 'emotions_encoding'] = new_column_value